## dataLoad

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import os
import random
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
from utils import custom_train_test_split, lgbm_predict, post_slack
base_feats = ['KnowledgeTag', 'user_correct_answer', 'user_total_answer', 
         'user_acc', 'test_mean', 'test_sum', 'tag_mean','tag_sum']
# train과 test 데이터셋은 사용자 별로 묶어서 분리를 해주어야함

## 1. 데이터 로딩
data_dir = '/opt/ml/input/data' # 경로
# train_file_path = os.path.join(data_dir, 'train_data.csv') # 데이터
# test_file_path = os.path.join(data_dir, 'test_data.csv')
after_fe_path = os.path.join(data_dir, 'after_fe_train_test_solvetime_user_test.pkl')
df = pd.read_pickle(after_fe_path)

train_df = df[df.kind=='train']

train, valid = custom_train_test_split(train_df, ratio=0.7, seed=13) # 훈련데이터 split
test = df[df.kind=='test'] # 테스트 데이터
train2 = test[test.answerCode!=-1] # 테스트데이터 마지막 제출 2번쨰꺼까지 훈련데이터로 사용
train = pd.concat([train_df,train2]) # 훈련데이터 병합
train.shape, valid.shape, test.shape

x_train = train.drop('answerCode',axis=1)
y_train = train[['answerCode']]

x_valid = valid.drop('answerCode',axis=1)
y_valid = valid[['answerCode']]
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

((2525956, 31), (2525956, 1), (1974, 31), (1974, 1))

## 문제푼 시간 초로만 추가한 것 (기존)

In [5]:
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model2 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model2.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

post_slack("done")

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.238892 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.582419
[200]	valid_0's binary_logloss: 0.575168
[300]	valid_0's binary_logloss: 0.568893
[400]	valid_0's binary_logloss: 0.563229
[500]	valid_0's binary_logloss: 0.559829
[600]	valid_0's binary_logloss: 0.556637
[700]	valid_0's binary_logloss: 0.553144
[800]	valid_0's binary_logloss: 0.550667
[900]	valid_0's binary_logloss: 0.548314
[1000]	valid_0's binary_logloss: 0.546135
[1100]	valid_0's binary_logloss: 0.542614
[1200]	valid_0's binary_logloss: 0.540631
[1300

### validation

In [169]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec'
        ]
preds = model2.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

VALID AUC : 0.8187398163235076 ACC : 0.7406281661600811



## 문제푼 시간 초로만 추가 (NaN 채우지 않음)

In [2]:
x_train[:2]

,userID,assessmentItemID,testId,Timestamp,KnowledgeTag,solvesec,solvesec_3600,time_category,user_correct_answer,user_total_answer,...,big_category,problem_num,mid_category,test_mean,test_std,test_sum,tag_mean,tag_std,tag_sum,kind
0,0,A060001001,A060000001,2020-03-24 00:17:11,7224,NaN,0.0,0,NaN,0,...,6,1,1,0.947683,0.222749,1268,0.955022,0.207410,637,train
1,0,A060001002,A060000001,2020-03-24 00:17:14,7225,3.0,3.0,1,1.0,1,...,6,2,1,0.947683,0.222749,1268,0.913187,0.281603,3040,train


In [4]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec'
        ]

params = {
#     "max_depth": 8,  # 8,
#     "min_data_in_leaf": 1000,
    # "feature_fraction": 0.6,  # 0.8,
#     "bagging_fraction": 0.75,
    # "max_cat_group": 64,
    "objective": "binary",
#     "boosting": "gbdt",  # dart
#     "learning_rate": 0.01,  # 0.01,
    # "bagging_freq": 5,
    "seed": 42,
    # "max_bin": 50,
#     "num_leaves": 80,  # 40,
#     "metric": "auc",
}

In [4]:
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model3 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model3.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

post_slack("done")

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.099724 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.586764
[200]	valid_0's binary_logloss: 0.580913
[300]	valid_0's binary_logloss: 0.57342
[400]	valid_0's binary_logloss: 0.568314
[500]	valid_0's binary_logloss: 0.564522
[600]	valid_0's binary_logloss: 0.56078
[700]	valid_0's binary_logloss: 0.558609
[800]	valid_0's binary_logloss: 0.555498
[900]	valid_0's binary_logloss: 0.552048
[1000]	valid_0's binary_logloss: 0.548929
[1100]	valid_0's binary_log

## 문제푼 시간 3600초 이상 모두 3600으로 변환 (기존)

In [11]:
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model4 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model4.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

post_slack("done")

lgbm_predict(test, model4, FEATS, 'LGBM_Baseline_문제푼시간_추가_3600초이상모두3600으로.csv')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.052752 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.57308
[200]	valid_0's binary_logloss: 0.564389
[300]	valid_0's binary_logloss: 0.559807
[400]	valid_0's binary_logloss: 0.554239
[500]	valid_0's binary_logloss: 0.550381
[600]	valid_0's binary_logloss: 0.547333
[700]	valid_0's binary_logloss: 0.542663
[800]	valid_0's binary_logloss: 0.539448
[900]	valid_0's binary_logloss: 0.537293
[1000]	valid_0's binary_logloss: 0.533369
[1100]	valid_0's binary_lo

### validation

In [170]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600'
        ]
preds = model4.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')
# 제출 AUC: 0.7789

VALID AUC : 0.8199340838394311 ACC : 0.7396149949341438



In [49]:
lgbm_predict(test, model4, FEATS, 'LGBM_Baseline_문제푼시간_추가_3600초이상모두3600으로.csv')

writing prediction : output/LGBM_Baseline_문제푼시간_추가_3600초이상모두3600으로.csv


# userID, testID 같이 groupby한 경우

## 문제푼 시간 3600초 이상 모두 3600으로 (NaN 채우지 않음)

In [5]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600'
        ]

lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model5 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model5.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

post_slack("done")

lgbm_predict(test, model5, FEATS, 'LGBM_Baseline_문제푼시간_추가_3600_NaN채우지않음_user_test.csv')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.044387 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.589353
[200]	valid_0's binary_logloss: 0.578821
[300]	valid_0's binary_logloss: 0.57417
[400]	valid_0's binary_logloss: 0.569103
[500]	valid_0's binary_logloss: 0.563665
[600]	valid_0's binary_logloss: 0.560852
[700]	valid_0's binary_logloss: 0.558971
[800]	valid_0's binary_logloss: 0.55587
[900]	valid_0's binary_logloss: 0.552249
[1000]	valid_0's binary_logloss: 0.551044
[1100]	valid_0's binary_log

### fillna(0) 해보기

In [11]:
x_train.solvesec_3600 = x_train.solvesec_3600.fillna(0)

In [10]:
x_valid.solvesec_3600.info()

<class 'pandas.core.series.Series'>
Int64Index: 1974 entries, 744 to 2526699
Series name: solvesec_3600
Non-Null Count  Dtype  
--------------  -----  
1974 non-null   float64
dtypes: float64(1)
memory usage: 30.8 KB


In [12]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600'
        ]

lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model5_1 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model5_1.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

post_slack("done")

lgbm_predict(test, model5_1, FEATS, 'LGBM_Baseline_문제푼시간_추가_3600_fillna0_user_test.csv')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.054967 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.584019
[200]	valid_0's binary_logloss: 0.578371
[300]	valid_0's binary_logloss: 0.573579
[400]	valid_0's binary_logloss: 0.566298
[500]	valid_0's binary_logloss: 0.561036
[600]	valid_0's binary_logloss: 0.559448
[700]	valid_0's binary_logloss: 0.557682
[800]	valid_0's binary_logloss: 0.554892
[900]	valid_0's binary_logloss: 0.552933
[1000]	valid_0's binary_logloss: 0.550578
[1100]	valid_0's binary_l

In [13]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3200'
        ]

lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model6 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model6.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

post_slack("done")

lgbm_predict(test, model6, FEATS, 'LGBM_Baseline_문제푼시간_추가_3200_NaN채우지않음_user_test.csv')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.046535 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.586502
[200]	valid_0's binary_logloss: 0.57959
[300]	valid_0's binary_logloss: 0.572732
[400]	valid_0's binary_logloss: 0.567639
[500]	valid_0's binary_logloss: 0.562424
[600]	valid_0's binary_logloss: 0.55853
[700]	valid_0's binary_logloss: 0.556248
[800]	valid_0's binary_logloss: 0.553459
[900]	valid_0's binary_logloss: 0.551091
[1000]	valid_0's binary_logloss: 0.549598
[1100]	valid_0's binary_log

## 문제푼 시간 2400초 이상 모두 2400초 (NaN 채우지 않음)

In [14]:
x_train[:2]

,userID,assessmentItemID,testId,Timestamp,KnowledgeTag,solvesec,solvesec_3600,time_category,user_correct_answer,user_total_answer,...,tag_mean,tag_std,tag_sum,kind,solvesec_3200,solvesec400,solvesec_1800,solvesec_1200,solvesec_800,solvesec_600
0,0,A060001001,A060000001,2020-03-24 00:17:11,7224,NaN,0.0,9999,NaN,0,...,0.955022,0.207410,637,train,NaN,NaN,NaN,NaN,NaN,NaN
1,0,A060001002,A060000001,2020-03-24 00:17:14,7225,3.0,3.0,"1 - (0, 5]",1.0,1,...,0.913187,0.281603,3040,train,3.0,3.0,3.0,3.0,3.0,3.0


In [15]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec400'
        ]

lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model6 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model6.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

# post_slack("done")

lgbm_predict(test, model6, FEATS, 'LGBM_Baseline_문제푼시간_추가_2400_NaN채우지않음_user_test.csv')
# 제출 AUC: 0.7828

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.050236 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.589189
[200]	valid_0's binary_logloss: 0.582206
[300]	valid_0's binary_logloss: 0.576758
[400]	valid_0's binary_logloss: 0.573431
[500]	valid_0's binary_logloss: 0.56895
[600]	valid_0's binary_logloss: 0.561961
[700]	valid_0's binary_logloss: 0.556918


KeyboardInterrupt: 

### fillna(0) 해보기

In [16]:
x_train.solvesec400 = x_train.solvesec400.fillna(0)

In [26]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec400'
        ]

lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model6_1 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model6_1.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

# post_slack("done")

lgbm_predict(test, model6_1, FEATS, 'LGBM_Baseline_문제푼시간_추가_2400_fillna0_user_test.csv')
# 제출 AUC: 0.7828

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.059677 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.586169
[200]	valid_0's binary_logloss: 0.579944
[300]	valid_0's binary_logloss: 0.5732
[400]	valid_0's binary_logloss: 0.568215
[500]	valid_0's binary_logloss: 0.563044
[600]	valid_0's binary_logloss: 0.558819
[700]	valid_0's binary_logloss: 0.556577
[800]	valid_0's binary_logloss: 0.554006
[900]	valid_0's binary_logloss: 0.551461
[1000]	valid_0's binary_logloss: 0.548987
[1100]	valid_0's binary_log

## 문제푼 시간 1800초 이상 모두 1800초 (Fillna 0)

In [27]:
x_train.solvesec_1800 = x_train.solvesec_1800.fillna(0)

In [28]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_1800'
        ]

lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model8 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model8.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

# post_slack("done")

lgbm_predict(test, model8, FEATS, 'LGBM_Baseline_문제푼시간_추가_1800_fillna0_user_test.csv')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.060570 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.586733
[200]	valid_0's binary_logloss: 0.578531
[300]	valid_0's binary_logloss: 0.572469
[400]	valid_0's binary_logloss: 0.567673
[500]	valid_0's binary_logloss: 0.562649
[600]	valid_0's binary_logloss: 0.558864
[700]	valid_0's binary_logloss: 0.556945
[800]	valid_0's binary_logloss: 0.553692
[900]	valid_0's binary_logloss: 0.552532
[1000]	valid_0's binary_logloss: 0.548963
[1100]	valid_0's binary_l

## 문제푼 시간 1200초 이상 모두 1200초 (Fillna 0)

In [29]:
x_train.solvesec_1200 = x_train.solvesec_1200.fillna(0)

In [30]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_1200'
        ]

lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model9 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model9.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

# post_slack("done")

lgbm_predict(test, model9, FEATS, 'LGBM_Baseline_문제푼시간_추가_1200_fillna0_user_test.csv')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.055768 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.586733
[200]	valid_0's binary_logloss: 0.578855
[300]	valid_0's binary_logloss: 0.573703
[400]	valid_0's binary_logloss: 0.568943
[500]	valid_0's binary_logloss: 0.564133
[600]	valid_0's binary_logloss: 0.556624
[700]	valid_0's binary_logloss: 0.553693
[800]	valid_0's binary_logloss: 0.552816
[900]	valid_0's binary_logloss: 0.549696
[1000]	valid_0's binary_logloss: 0.547725
[1100]	valid_0's binary_l

## 문제푼 시간 800초 이상 모두 800초 (Fillna 0)

In [31]:
x_train.solvesec_800 = x_train.solvesec_800.fillna(0)

In [32]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_800'
        ]

lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model10 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model10.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

# post_slack("done")

lgbm_predict(test, model10, FEATS, 'LGBM_Baseline_문제푼시간_추가_800_fillna0_user_test.csv')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.051280 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.585567
[200]	valid_0's binary_logloss: 0.577478
[300]	valid_0's binary_logloss: 0.571909
[400]	valid_0's binary_logloss: 0.566423
[500]	valid_0's binary_logloss: 0.561582
[600]	valid_0's binary_logloss: 0.557073
[700]	valid_0's binary_logloss: 0.554236
[800]	valid_0's binary_logloss: 0.552168
[900]	valid_0's binary_logloss: 0.549849
[1000]	valid_0's binary_logloss: 0.547453
[1100]	valid_0's binary_l

## 문제푼 시간 600초 이상 모두 600초 (Fillna 0)

In [33]:
x_train.solvesec_600 = x_train.solvesec_600.fillna(0)

In [34]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_600'
        ]

lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model11 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model11.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

post_slack("done")

lgbm_predict(test, model11, FEATS, 'LGBM_Baseline_문제푼시간_추가_600_fillna0_user_test.csv')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.059873 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.585035
[200]	valid_0's binary_logloss: 0.577606
[300]	valid_0's binary_logloss: 0.573044
[400]	valid_0's binary_logloss: 0.567491
[500]	valid_0's binary_logloss: 0.563481
[600]	valid_0's binary_logloss: 0.559263
[700]	valid_0's binary_logloss: 0.557925
[800]	valid_0's binary_logloss: 0.555063
[900]	valid_0's binary_logloss: 0.55208
[1000]	valid_0's binary_logloss: 0.549163
[1100]	valid_0's binary_lo

## 문제푼 시간 카테고리 변수도 추가 (기존)

In [16]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec',
         'time_category'
        ]

In [17]:
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model5 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model5.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.052055 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3105
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.560899
[200]	valid_0's binary_logloss: 0.552517
[300]	valid_0's binary_logloss: 0.547221
[400]	valid_0's binary_logloss: 0.540192
[500]	valid_0's binary_logloss: 0.535174
[600]	valid_0's binary_logloss: 0.531541
[700]	valid_0's binary_logloss: 0.527031
[800]	valid_0's binary_logloss: 0.523995
[900]	valid_0's binary_logloss: 0.522249
[1000]	valid_0's binary_logloss: 0.519503
[1100]	valid_0's binary_l

### validation

In [171]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600',
         'time_category'
        ]
preds = model5.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

VALID AUC : 0.8215223259105646 ACC : 0.7406281661600811



In [18]:
post_slack("done")

## 문제푼 시간 2400 초 이상, 카테고리 변수 추가

In [21]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_2400_2',
         'time_category'
        ]

lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model13 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model13.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

post_slack("done")

lgbm_predict(test, model13, FEATS, 'LGBM_Baseline_문제푼시간_추가_2400_NaN채우지않음_timecategory추가.csv')
# 제출 AUC: 0.7828 -> 

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.049040 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3105
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.588256
[200]	valid_0's binary_logloss: 0.581898
[300]	valid_0's binary_logloss: 0.573774
[400]	valid_0's binary_logloss: 0.568879
[500]	valid_0's binary_logloss: 0.561
[600]	valid_0's binary_logloss: 0.558781
[700]	valid_0's binary_logloss: 0.554875
[800]	valid_0's binary_logloss: 0.55249
[900]	valid_0's binary_logloss: 0.550776
[1000]	valid_0's binary_logloss: 0.549105
[1100]	valid_0's binary_loglo

In [52]:
lgbm_predict(test, model5, FEATS, 'LGBM_Baseline_문제푼시간_추가_최대3600으로_time_category추가.csv')

writing prediction : output/LGBM_Baseline_문제푼시간_추가_최대3600으로_time_category추가.csv
